# Prevendo roubo de celulares em SP

## Baixando biblioteca necessária
`pip install xlwt`

`conda install xlwt`

## Importando bibliotecas Pandas e Numpy

In [1]:
import pandas as pd
import numpy as np

## Abrindo arquivo XLS disponível no SSP-SP

### Os arquivos possuem um erro, sendo impossível abrir o arquivo
*XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'A\x00N\x00O\x00_\x00'*

A solução foi encontrada nesse artigo do [Medium](https://medium.com/@jerilkuriakose/recover-corrupt-excel-file-xls-or-xlsx-using-python-2eea6bb07aae)

#### Apenas para resolver o problema de formato
Você pode até salvar o arquivo como .csv, mas a leitura pode informar algum erro

In [8]:
from __future__ import unicode_literals
# Used to save the file as excel workbook
# Need to install this library
from xlwt import Workbook
# Used to open to corrupt excel file
import io

filename = 'datasets/DadosBO_2020_8.xls'
# Opening the file using 'utf-16' encoding
file = io.open(filename, "r", encoding="utf-16-le")
data = file.readlines()

# Creating a workbook object
xldoc = Workbook()
# Adding a sheet to the workbook object
sheet = xldoc.add_sheet("Sheet1", cell_overwrite_ok=True)
# Iterating and saving the data to sheet
for i, row in enumerate(data):
    # Two things are done here
    # Removeing the '\n' which comes while reading the file using io.open
    # Getting the values after splitting using '\t'
    for j, val in enumerate(row.replace('\n', '').split('\t')):
        sheet.write(i, j, val)
    
# Saving the file as an excel file
xldoc.save('datasets/file.xls')

#### Converter xls para csv

In [12]:
import xlrd
import csv

def xls_to_csv():
    wb = xlrd.open_workbook('datasets/file.xls')
    sh = wb.sheet_by_name('Sheet1')
    with open('dataset.csv', 'w') as f:
        wr = csv.writer(f, quoting=csv.QUOTE_ALL)

        for rownum in range(sh.nrows):
            wr.writerow(sh.row_values(rownum))
#https://stackoverflow.com/questions/9884353/xls-to-csv-converter
xls_to_csv()

## Carregando o dataset

In [47]:
#dataset = pd.read_excel("DadosBO_2020_7.xls", engine='xlrd')
columns = ['ANO_BO', 'NUM_BO', 'BO_INICIADO', 'BO_EMITIDO', 'DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA', 'DATACOMUNICACAO', 'DATAELABORACAO', 'BO_AUTORIA', 'FLAGRANTE', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE', 'UF', 'LATITUDE', 'LONGITUDE', 'DESCRICAOLOCAL', 'SOLUCAO', 'DELEGACIA_NOME', 'DELEGACIA_CIRCUNSCRICAO', 'ESPECIE', 'RUBRICA', 'DESDOBRAMENTO', 'STATUS', 'QUANT_CELULAR', 'MARCA_CELULAR']

column_dates = ['BO_INICIADO', 'BO_EMITIDO', 'DATAOCORRENCIA','DATACOMUNICACAO', 'DATAELABORACAO']

dtypes = {
    'ANO_BO': np.int32,
    'NUM_BO': np.int64,
    'QUANT_CELULAR': pd.Int16Dtype(),
    'LATITUDE': np.float32,
    'LONGITUDE': np.float32 
}

dataset = pd.read_excel("datasets/file.xls", parse_dates=column_dates, usecols=columns, dtype=dtypes, thousands=',')

In [85]:
dataset.dtypes

ANO_BO                              int32
NUM_BO                              int64
BO_INICIADO                datetime64[ns]
BO_EMITIDO                 datetime64[ns]
DATAOCORRENCIA             datetime64[ns]
HORAOCORRENCIA             datetime64[ns]
PERIDOOCORRENCIA                   object
DATACOMUNICACAO            datetime64[ns]
DATAELABORACAO             datetime64[ns]
BO_AUTORIA                         object
FLAGRANTE                          object
LOGRADOURO                         object
NUMERO                            float64
BAIRRO                             object
CIDADE                             object
UF                                 object
LATITUDE                          float32
LONGITUDE                         float32
DESCRICAOLOCAL                     object
SOLUCAO                            object
DELEGACIA_NOME                     object
DELEGACIA_CIRCUNSCRICAO            object
ESPECIE                            object
RUBRICA                           

### Análise gráfica dos dados

#### Capturando nome das colunas importantes no dataset

In [39]:
string_gerada=""
for item in dataset.columns:
    string_gerada += "'"+item+"'"+", "
string_gerada

"'ANO_BO', 'NUM_BO', 'NUMERO_BOLETIM', 'BO_INICIADO', 'BO_EMITIDO', 'DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA', 'DATACOMUNICACAO', 'DATAELABORACAO', 'BO_AUTORIA', 'FLAGRANTE', 'NUMERO_BOLETIM_PRINCIPAL', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE', 'UF', 'LATITUDE', 'LONGITUDE', 'DESCRICAOLOCAL', 'EXAME', 'SOLUCAO', 'DELEGACIA_NOME', 'DELEGACIA_CIRCUNSCRICAO', 'ESPECIE', 'RUBRICA', 'DESDOBRAMENTO', 'STATUS', 'NOMEPESSOA', 'TIPOPESSOA', 'VITIMAFATAL', 'RG', 'RG_UF', 'NATURALIDADE', 'NACIONALIDADE', 'SEXO', 'DATANASCIMENTO', 'IDADE', 'ESTADOCIVIL', 'PROFISSAO', 'GRAUINSTRUCAO', 'CORCUTIS', 'NATUREZAVINCULADA', 'TIPOVINCULO', 'RELACIONAMENTO', 'PARENTESCO', 'PLACA_VEICULO', 'UF_VEICULO', 'CIDADE_VEICULO', 'DESCR_COR_VEICULO', 'DESCR_MARCA_VEICULO', 'ANO_FABRICACAO', 'ANO_MODELO', 'DESCR_TIPO_VEICULO', 'QUANT_CELULAR', 'MARCA_CELULAR', "

In [42]:
drop_c = ['ANO_BO', 'NUM_BO', 'BO_INICIADO', 'BO_EMITIDO', 'DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA', 'DATACOMUNICACAO', 'DATAELABORACAO', 'BO_AUTORIA', 'FLAGRANTE', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE', 'UF', 'LATITUDE', 'LONGITUDE', 'DESCRICAOLOCAL', 'SOLUCAO', 'DELEGACIA_NOME', 'DELEGACIA_CIRCUNSCRICAO', 'ESPECIE', 'RUBRICA', 'DESDOBRAMENTO', 'STATUS', 'QUANT_CELULAR', 'MARCA_CELULAR']

In [ ]:
string_gerada=[]
for item in dataset.columns:
    string_gerada.append(item)
string_gerada

In [47]:
columns = ['NUMERO_BOLETIM', 'NUMERO_BOLETIM_PRINCIPAL', 'EXAME', 'NOMEPESSOA', 'TIPOPESSOA', 'VITIMAFATAL', 'RG', 'RG_UF', 'NATURALIDADE', 'NACIONALIDADE', 'SEXO', 'DATANASCIMENTO', 'IDADE', 'ESTADOCIVIL', 'PROFISSAO', 'GRAUINSTRUCAO', 'CORCUTIS', 'NATUREZAVINCULADA', 'TIPOVINCULO', 'RELACIONAMENTO', 'PARENTESCO', 'PLACA_VEICULO', 'UF_VEICULO', 'CIDADE_VEICULO', 'DESCR_COR_VEICULO', 'DESCR_MARCA_VEICULO', 'ANO_FABRICACAO', 'ANO_MODELO', 'DESCR_TIPO_VEICULO']

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
